# Train gradient boosting model using Bayesian optimization to find best hyperparameters

# 1. Imports

## 1.1 Packages

In [1]:
import sys

import pandas as pd


## 1.2 Options

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../src')

# from velib_prediction.pipelines.train_model.mlflow import (  # noqa: E402
#     create_mlflow_experiment,
# )
from velib_prediction.pipelines.train_model.nodes import (  # noqa: E402
    add_lags_sma,
    # get_split_train_val_cv,
    split_train_valid_last_hours,
    train_model_bayesian_opti,
)
    # train_model_mlflow,
# )


In [4]:
lags_to_try = [1,]

In [5]:
feat_date = "duedate"

## 1.3 Datasets

In [6]:
df_training = pd.read_parquet("../data/04_feature/df_feat_train.parquet")
df_training.sample(2)

,idx,stationcode,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
5,152021730671980,15202,1,30,23,6,3,3,1,1,2024-11-03 22:13:00+00:00,75056,2024,11,3,6,1
1,60031730613082,6003,1,21,3,17,13,4,1,1,2024-11-03 05:51:22+00:00,75056,2024,11,3,6,1


In [7]:
df_training.rename(columns={"numbikesavailable": "target"}, inplace=True)

In [8]:
df_training.tail()

,idx,stationcode,is_installed,capacity,numdocksavailable,target,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
5,170411729666463,17041,1,36,28,8,8,0,1,1,2024-10-23 06:54:23+00:00,75056,2024,10,23,2,0
6,150471729666488,15047,1,52,48,2,2,0,1,1,2024-10-23 06:54:48+00:00,75056,2024,10,23,2,0
7,121091729666278,12109,1,30,22,8,6,2,1,1,2024-10-23 06:51:18+00:00,75056,2024,10,23,2,0
8,141081729666376,14108,1,21,17,3,2,1,1,1,2024-10-23 06:52:56+00:00,75056,2024,10,23,2,0
9,161381729666193,16138,1,30,15,14,10,4,1,1,2024-10-23 06:49:53+00:00,75056,2024,10,23,2,0


# 2. Prepare datasets

In [9]:
# Add lags defined
df_training = add_lags_sma(df_training, lags_to_try, feat_id='stationcode', feat_date=feat_date, feat_target="target", n_shift=5)

In [10]:
df_training.sort_values(by="duedate", inplace=True)

In [11]:
# df_training.drop(columns="duedate", inplace=True)
# list_df = get_split_train_val_cv(df_training, n_splits=2)

# df_train, df_valid = list_df[0][0], list_df[0][1]

In [12]:
df_training.loc[df_training["stationcode"] == '11025']

,idx,stationcode,is_installed,capacity,numdocksavailable,target,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend,sma_1_lag
7,110251729352699,11025,1,43,15,26,15,11,1,1,2024-10-19 15:44:59+00:00,75056,2024,10,19,5,1,NaN
6,110251729356310,11025,1,43,17,22,17,5,1,1,2024-10-19 16:45:10+00:00,75056,2024,10,19,5,1,NaN
5,110251729363427,11025,1,43,9,30,23,7,1,1,2024-10-19 18:43:47+00:00,75056,2024,10,19,5,1,NaN
6,110251729377872,11025,1,43,7,34,28,6,1,1,2024-10-19 22:44:32+00:00,75056,2024,10,19,5,1,NaN
6,110251729388654,11025,1,43,5,35,27,8,1,1,2024-10-20 01:44:14+00:00,75056,2024,10,20,6,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,110251730656452,11025,1,43,14,29,21,8,1,1,2024-11-03 17:54:12+00:00,75056,2024,11,3,6,1,28.0
6,110251730657905,11025,1,43,5,38,27,11,1,1,2024-11-03 18:18:25+00:00,75056,2024,11,3,6,1,21.0
5,110251730710732,11025,1,43,11,31,27,4,1,1,2024-11-04 08:58:52+00:00,75056,2024,11,4,0,0,14.0
9,110251730721334,11025,1,43,13,30,27,3,1,1,2024-11-04 11:55:34+00:00,75056,2024,11,4,0,0,28.0


In [13]:
df_train, df_valid = split_train_valid_last_hours(df_training, n_hours=24)

In [14]:
feat_cat = [
    "is_installed",
    "is_renting",
    "is_returning",
    "code_insee_commune",
    "duedate_weekend",
]

# 3. Train model

In [15]:
# experiment_id = create_mlflow_experiment(
#     experiment_folder_path="../data/06_models/mlruns",
#     experiment_name="velib_prediction"
# )
# experiment_id

experiment_id = '587291553688351204'

In [16]:
search_params = {
    "iterations": {
        "min": 100,
        "max": 500,
        "sampling_type": "int",
    },
    "depth": {
        "min": 2,
        "max": 7,
        "sampling_type": "int",
    },
    "learning_rate": {
        "min": 0.0001,
        "max": 0.5,
        "sampling_type": "loguniform"
    },
}

# params = {
#     "iterations": 100,
#     "depth": 4
# }

In [17]:
cols_to_drop = [
    "idx", "duedate",
]

In [18]:
df_train.drop(columns=cols_to_drop)

,stationcode,is_installed,capacity,numdocksavailable,target,mechanical,ebike,is_renting,is_returning,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend,sma_1_lag
6,6108,1,17,11,6,5,1,1,1,75056,2024,10,19,5,1,NaN
1,9020,1,21,13,7,4,3,1,1,75056,2024,10,19,5,1,NaN
1,9020,1,21,13,7,4,3,1,1,75056,2024,10,19,5,1,NaN
2,14111,1,25,21,2,1,1,1,1,75056,2024,10,19,5,1,NaN
7,15202,1,30,26,4,1,3,1,1,75056,2024,10,19,5,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,14014,1,60,45,12,5,7,1,1,75056,2024,11,3,6,1,17.0
4,6003,1,21,2,18,12,6,1,1,75056,2024,11,3,6,1,17.0
3,6003,1,21,4,16,13,3,1,1,75056,2024,11,3,6,1,17.0
3,8026,1,12,9,2,1,1,1,1,75056,2024,11,3,6,1,3.0


In [19]:
train_model_bayesian_opti(
    run_name="Test_catboost",
    experiment_id=experiment_id,
    search_params=search_params,
    df_train=df_train.drop(columns=cols_to_drop),
    df_valid=df_valid.drop(columns=cols_to_drop),
    feat_cat=feat_cat,
    n_trials=10
)

# model, rmse_train, rmse_valid = train_model_mlflow(
#     experiment_id=experiment_id,
#     parent_run_id=None,
#     df_train=df_train,
#     df_valid=df_valid,
#     feat_cat=feat_cat,
#     **params
# )

# print("RMSE train:", rmse_train)
# print("RMSE valid:", rmse_valid)

[I 2024-12-09 08:01:50,852] A new study created in memory with name: 


  0%|          | 0/10 [00:00<?, ?it/s]

<string>:1: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings

0:	learn: 9.5568163	test: 8.4866391	best: 8.4866391 (0)	total: 59.2ms	remaining: 13s
100:	learn: 9.1623635	test: 8.1718421	best: 8.1718421 (100)	total: 126ms	remaining: 150ms
200:	learn: 8.7861010	test: 7.8816464	best: 7.8816464 (200)	total: 199ms	remaining: 19.8ms
220:	learn: 8.7129440	test: 7.8238123	best: 7.8238123 (220)	total: 212ms	remaining: 0us

bestTest = 7.82381234
bestIteration = 220



<string>:1: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings

...

<string>:1: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings

262:	learn: 9.2496899	test: 8.2435344	best: 8.2435344 (262)	total: 179ms	remaining: 0us

bestTest = 8.243534426
bestIteration = 262

[I 2024-12-09 08:02:04,812] Trial 8 finished with value: 8.243534425601819 and parameters: {'iterations': 263, 'depth': 7, 'learning_rate': 0.00015078459524436603}. Best is trial 8 with value: 8.243534425601819.
0:	learn: 9.2074235	test: 8.1887186	best: 8.1887186 (0)	total: 596us	remaining: 253ms
100:	learn: 0.5292652	test: 1.6838851	best: 1.6838851 (100)	total: 49.4ms	remaining: 159ms
200:	learn: 0.2715914	test: 1.4847505	best: 1.4847505 (200)	total: 97.2ms	remaining: 108ms
300:	learn: 0.2048121	test: 1.4549661	best: 1.4547176 (280)	total: 146ms	remaining: 60ms


<string>:1: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings

400:	learn: 0.1570922	test: 1.4454197	best: 1.4446821 (391)	total: 194ms	remaining: 11.6ms
424:	learn: 0.1485193	test: 1.4435872	best: 1.4435398 (418)	total: 211ms	remaining: 0us

bestTest = 1.443539816
bestIteration = 418

Shrink model to first 419 iterations.
[I 2024-12-09 08:02:06,317] Trial 9 finished with value: 1.4435398162573434 and parameters: {'iterations': 425, 'depth': 5, 'learning_rate': 0.0434421882674713}. Best is trial 8 with value: 8.243534425601819.


{'iterations': 263, 'depth': 7, 'learning_rate': 0.00015078459524436603}

In [1]:
import pandas as pd

df = pd.read_parquet("../data/04_feature/df_feat_train.parquet")

df.head(1000).to_parquet("../tests/data/df_train.parquet")

,idx,stationcode,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
0,90201730580768,9020,1,21,13,8,0,8,1,1,2024-11-02 20:52:48+00:00,75056,2024,11,2,5,1
1,141111730580684,14111,1,25,18,3,0,3,1,1,2024-11-02 20:51:24+00:00,75056,2024,11,2,5,1
2,140141730580750,14014,1,60,47,11,2,9,1,1,2024-11-02 20:52:30+00:00,75056,2024,11,2,5,1
3,60031730580840,6003,1,21,1,20,10,10,1,1,2024-11-02 20:54:00+00:00,75056,2024,11,2,5,1
4,61081730580670,6108,1,17,12,4,1,3,1,1,2024-11-02 20:51:10+00:00,75056,2024,11,2,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,61081729796238,6108,1,17,13,4,3,1,1,1,2024-10-24 18:57:18+00:00,75056,2024,10,24,3,0
6,70031729796295,7003,1,60,55,2,2,0,1,1,2024-10-24 18:58:15+00:00,75056,2024,10,24,3,0
7,110251729796230,11025,1,43,6,31,20,11,1,1,2024-10-24 18:57:10+00:00,75056,2024,10,24,3,0
8,150471729796149,15047,1,52,42,7,3,4,1,1,2024-10-24 18:55:49+00:00,75056,2024,10,24,3,0
